In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from envs.CarRacing import CarRacing
from networks.PolicyNet import PolicyNetImage
from networks.ValueNet import ValueNetImage
from memory.RewardMemory import Memory
from tqdm.notebook import tqdm
import numpy as np
import sys
import traceback
from networks.utils import *
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
env = CarRacing()
critic = ValueNetImage(input_channels=3, hidden_dim=128)
actor = PolicyNetImage(input_channels=3, hidden_dim=128, action_dims=3)

memory = Memory(3, env.states, 3, 15000)

sigma = 1.0
_lambda = 0.97
gamma = 0.99
batch_size = 256

rewards_per_ep = {} # store rewards for each episode

for e in tqdm(range(500)):
  state = env.reset()
  cont = 0
  ep_reward = 0

  for t in range(1000):
    action, logp = actor(state)
    action_dt = action.cpu().detach().numpy().reshape(3)
    #print(action_dt)
    logp_dt = logp.cpu().detach()
    # print(action_np.shape)
    #print(type(action_np))

    v = critic(state)
    v_dt = v.cpu().detach()

    obs, r, terminal, truncated, info = env.step(action_dt)
    ep_reward += r
    if action_dt[1] > 0.5:
       ep_reward += 0.1

    memory.add(state, action_dt, r, obs, terminal, v_dt, logp_dt)
    state = np.copy(obs)

    if terminal or truncated:
      break

  rewards_per_ep[e] = [ep_reward, cont]

  if terminal:
      v = 0
  else:
      v = critic(obs)
      
  v_dt = v.cpu().detach()

  i = len(memory)
  memory.adv[i] = 0
  memory.rtg[i] = v_dt

  for i in reversed(range(len(memory)-t, len(memory))):
      delta = memory.r[i] + gamma*memory.v[i+1]-memory.v[i]
      adv = gamma*_lambda*memory.adv[i+1] + delta
      rtg = gamma*memory.rtg[i+1]+memory.r[i]
      
      memory.adv[i] = adv
      memory.rtg[i] = rtg
      
  if e % 10 == 0 and e != 0:
      used = slice(0, len(memory))
      for i in tqdm(range(80), leave=False):
          # update(self, s, a_prev, logp_a, adv, clip_ratio=0.2)
          #s, a, r, sp, terminal, v=0, logp=0
          actor.update(memory.s[used], memory.a[used], memory.logp[used], memory.adv[used])
          # update(self, inputs, targets)
          critic.update(memory.s[used], memory.rtg[used])
      
      #tqdm.write(f"Ep {e}: Actor and Critic updated | Ep reward: {ep_reward} | Last 100 rewards: {np.mean(list(rewards_per_ep.values())[-100:], axis=0)[0]}")
      memory.reset()



    

# Close environment
#env.plotnetwork(critic, actor)
env.close()

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

c:\Users\jpkqv\anaconda3\envs\rlga\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
env.close()

In [4]:
# Save models
actor.save("models")
critic.save("models")